In [4]:
import json
import pandas as pd
import requests
import time

ADDRESS = 'TYDzsYUEpvnYmQk4zGP9sWWcTEd2MiAtW6'
INITIAL_TIMESTAMP = 1619403603000
LIMIT = 199

intial_url = f'https://api.trongrid.io/v1/accounts/{ADDRESS}/transactions/trc20'

headers = {
    'Content-Type': "application/json",
    'TRON-PRO-API-KEY': "8f6799a3-30ea-4d9a-a99b-6d9fa9b74730",
}

r = requests.get(
        intial_url, 
        headers=headers, 
        params={
            "limit":LIMIT,
            "max_timestamp": INITIAL_TIMESTAMP,
        }
)
json.loads(r.content)['meta']

{'at': 1619403630678,
 'fingerprint': '3wmN1yM3XgcVSRuqUempoB9SSddnDfigkc2j1FcjttR4umd3G7BdSUTRr9Fu9t6H2DwJDjKifY2dcwAwJzKXEHzj2sBMTLYLnUhq',
 'links': {'next': 'https://api.trongrid.io/v1/accounts/TYDzsYUEpvnYmQk4zGP9sWWcTEd2MiAtW6/transactions/trc20?limit=199&max_timestamp=1619403603000&fingerprint=3wmN1yM3XgcVSRuqUempoB9SSddnDfigkc2j1FcjttR4umd3G7BdSUTRr9Fu9t6H2DwJDjKifY2dcwAwJzKXEHzj2sBMTLYLnUhq'},
 'page_size': 199}

In [5]:
res = []
for i in range(20_000):
    try:
        rs = json.loads(r.content)
        res.append(rs)
        # get last time due to stupid API restriction
        this_time = rs['data'][-1]['block_timestamp']
        for d in rs['data']:
            if d['block_timestamp'] < this_time:
                this_time = d['block_timestamp']
        # Get next part of the response
        r = requests.get(
                intial_url, 
                headers=headers, 
                params={
                    "limit":LIMIT,
                    "max_timestamp": this_time,
                    "min_timestamp": this_time - 1_000_000_000,
                }
        )
    except KeyError:
        break

print(len(res))

# Save intermediate JSON result object
j = json.dumps(res, indent=4)
with open("ftx.json", 'w') as f:
    f.write(j)

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [ ]:
# Load JSON and convert transactions to df
with open("ftx.json", "r") as f:
    j = json.load(f)

df = pd.DataFrame(sum([a['data'] for a in j], []))
df.to_csv("../ftx.csv", index=False)
df.head()